# $ES(\mu$, $\lambda)$ and $ES(\mu+\lambda)$ algorithms 

In [1]:
import numpy as np
import wandb
from tqdm.notebook import tqdm

In [2]:
class ES_search:
    problem_dimensionality: int
    parents_number: int
    descendants_number: int
    tau: float
    tau_zero: float

    def __init__(self, problem_dimensionality, parents_number, initial_min, initial_max, descendants_number, tau, tau_zero, fitness_function,
                 sum_populations=True, ranking_resampling=True, **args) -> None:
        self.problem_dimensionality = problem_dimensionality
        self.parents_number = parents_number
        self.descendants_number = descendants_number
        self.tau = tau
        self.tau_zero = tau_zero
        self.fitness_function = fitness_function
        self.sum_populations = sum_populations
        self.initial_min = initial_min
        self.initial_max = initial_max
        self.ranking_resampling = ranking_resampling
        self.best_chromosome = None

        self.initialize_population()

    def initialize_population(self):
        genes = np.random.uniform(self.initial_min, self.initial_max, (self.parents_number, self.problem_dimensionality)) 
        mutation_rates = np.random.uniform(self.initial_min, self.initial_max, (self.parents_number, self.problem_dimensionality)) 
        self.population = (genes, mutation_rates)

    def evaluate_population(self, generation):
        genes, mutation_rates = self.population
        fitness = np.array([self.fitness_function(gene) for gene in genes])
        wandb.log({"generation": generation, "fitness_max": fitness.max(), "fitness_min": fitness.min(), "fitness_avg": fitness.mean()})
        if not self.best_chromosome or fitness.max() > self.best_chromosome['fitness']:
            best_id = np.argmax(fitness.max())
            self.best_chromosome = {
                'fitness': fitness.max(),
                'genes': genes[best_id],
                'mutation_rates': mutation_rates[best_id],
                'generation': generation
                                   }
            wandb.log({"best_chromosome": self.best_chromosome})
        return fitness

    def evolve(self, max_generations):
        self.evaluate_population(0)
        for generation in range(max_generations):
            fitness = self.evolve_generation(generation)
            print(f"Generation {generation} Max fitness: {fitness.max()} Min fitness: {fitness.min()} Avg fitness: {fitness.mean()}")

    def select_parents(self, genes, mutation_rates, fitness):
        indices_to_be_mutated = np.random.choice(self.parents_number, self.descendants_number, replace=False)
        return genes[indices_to_be_mutated], mutation_rates[indices_to_be_mutated]

    def evolve_generation(self, generation):
        genes, mutation_rates = self.population
        fitness = self.evaluate_population(generation)
        genes_to_be_mutated, mutation_rates_to_be_mutated = self.select_parents(genes, mutation_rates, fitness)
        descendant_genes, descendant_mutation_rates = self.generate_descendants(genes_to_be_mutated, mutation_rates_to_be_mutated)

        self.population = self.recombine_populations(genes, mutation_rates, fitness, descendant_genes, descendant_mutation_rates)

        return fitness

    def generate_descendants(self, genes, mutation_rates):
        epsilons = np.random.normal(0, self.tau**2, (self.descendants_number, self.problem_dimensionality))
        epsilons_zero = np.random.normal(0, self.tau_zero**2)
        descendant_genes = genes + epsilons
        descendant_mutation_rates = mutation_rates + np.exp(epsilons_zero + epsilons)
        return descendant_genes, descendant_mutation_rates
    
    def recombine_populations(self, parent_genes, parent_mutation_rates, parent_fitness, descendant_genes, descendant_mutation_rates):
        if self.sum_populations:
            genes = np.vstack([parent_genes, descendant_genes])
            mutation_rates = np.vstack([parent_mutation_rates, descendant_mutation_rates])
            fitness = np.concatenate([parent_fitness, np.array([self.fitness_function(gene) for gene in descendant_genes])])
            if self.ranking_resampling:
                indices = fitness.argsort()[-self.parents_number:]
                return genes[indices], mutation_rates[indices]
            else:
                indices = np.random.choice(self.parents_number+self.descendants_number, self.parents_number, replace=True, p=fitness/fitness.sum())
                return genes[indices], mutation_rates[indices]
        else:
            return descendant_genes, descendant_mutation_rates

## Benchmark functions

In [3]:
def griewank(x):
    fr = 4000
    s = (x**2).sum()
    p = np.array([np.cos(x_i/np.sqrt(i+1)) for i, x_i in enumerate(x)]).prod()
    y = s/fr - p + 1
    return y

In [4]:
def rastrigin(x):
    n = x.shape[0]
    s = np.sum([x_i**2-10*np.cos(2*np.pi*x_i) for x_i in x])
    y = 10*n+s
    return y

In [5]:
def schwefel(x):
    s = np.sum(-x*np.sin(np.sqrt(np.abs(x))))
    y = 418.9829*x.shape[0]+s
    return y

In [11]:
def michalewicz(x):
    m = 10
    s = np.sum([np.sin(x_i)*(np.sin(i*x_i**2/np.pi))**(2*m) for i, x_i in enumerate(x)])
    return -s

In [7]:
def sum_squares(x):
    s = np.sum([(i+1)*x_i**2 for i, x_i in enumerate(x)])
    return s

## Experiments

In [25]:
experiments = [
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -400,
    #     'initial_max': 400,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -griewank(x),
    #     'fitness_name': 'griewank',
    #     'generations': 1000,
    #     'known_minimum': 0,
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -5,
    #     'initial_max': 5,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -rastrigin(x),
    #     'fitness_name': 'rastrigin',
    #     'generations': 1000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 5,
    #     'parents_number': 2500,
    #     'initial_min': -5,
    #     'initial_max': 5,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -rastrigin(x),
    #     'fitness_name': 'rastrigin',
    #     'generations': 1000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 10,
    #     'parents_number': 2500,
    #     'initial_min': -5,
    #     'initial_max': 5,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -rastrigin(x),
    #     'fitness_name': 'rastrigin',
    #     'generations': 1000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -450,
    #     'initial_max': 450,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -schwefel(x),
    #     'fitness_name': 'schewefel',
    #     'generations': 1000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': 0.1,
    #     'initial_max': 0.9*np.pi,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -michalewicz(x),
    #     'fitness_name': 'michalewicz',
    #     'generations': 1000,
    #     'known_minimum': -1.8013
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -8,
    #     'initial_max': 8,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -sum_squares(x),
    #     'fitness_name': 'sum_squares',
    #     'generations': 1000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -450,
    #     'initial_max': 450,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -schwefel(x),
    #     'fitness_name': 'schewefel',
    #     'generations': 10000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': 0.1,
    #     'initial_max': 0.9*np.pi,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero':0.1,
    #     'fitness_function': lambda x: -michalewicz(x),
    #     'fitness_name': 'michalewicz',
    #     'generations': 10000,
    #     'known_minimum': -1.8013
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -400,
    #     'initial_max': 400,
    #     'descendants_number': 500,
    #     'tau': 0.1,
    #     'tau_zero': 0.1,
    #     'fitness_function': lambda x: -griewank(x),
    #     'fitness_name': 'griewank',
    #     'generations': 10000,
    #     'known_minimum': 0,
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': -450,
    #     'initial_max': 450,
    #     'descendants_number': 100,
    #     'tau': 0.5,
    #     'tau_zero':0.5,
    #     'fitness_function': lambda x: -schwefel(x),
    #     'fitness_name': 'schewefel',
    #     'generations': 5000,
    #     'known_minimum': 0
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': 0.1,
    #     'initial_max': 0.9*np.pi,
    #     'descendants_number': 500,
    #     'tau': 0.25,
    #     'tau_zero':0.25,
    #     'fitness_function': lambda x: -michalewicz(x),
    #     'fitness_name': 'michalewicz',
    #     'generations': 1000,
    #     'known_minimum': -1.8013
    # },
    # {
    #     'problem_dimensionality': 2,
    #     'parents_number': 2500,
    #     'initial_min': 0.1,
    #     'initial_max': 0.9*np.pi,
    #     'descendants_number': 100,
    #     'tau': 0.25,
    #     'tau_zero':0.25,
    #     'fitness_function': lambda x: -michalewicz(x),
    #     'fitness_name': 'michalewicz',
    #     'generations': 5000,
    #     'known_minimum': -1.8013
    # },
    {
        'problem_dimensionality': 2,
        'parents_number': 2500,
        'initial_min': -400,
        'initial_max': 400,
        'descendants_number': 100,
        'tau': 0.5,
        'tau_zero': 0.5,
        'fitness_function': lambda x: -griewank(x),
        'fitness_name': 'griewank',
        'generations': 10000,
        'known_minimum': 0,
    },
]

In [16]:
def run_experiment(config):
    es = ES_search(**config)
    wandb.init(config=config)
    es.evolve(config['generations'])

In [26]:
for experiment in tqdm(experiments):
    run_experiment(experiment)

  0%|          | 0/1 [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


fitness_avg,▁███████████████████████████████████████
fitness_max,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fitness_min,▁███████████████████████████████████████
generation,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
fitness_avg,0.8013
fitness_max,0.8013
fitness_min,0.8013
generation,4999


Generation 0 Max fitness: -0.268343921365924 Min fitness: -79.8764981969497 Avg fitness: -27.475336989991785
Generation 1 Max fitness: -0.268343921365924 Min fitness: -61.22015597773209 Avg fitness: -25.825675650869826
Generation 2 Max fitness: -0.268343921365924 Min fitness: -53.59016492243252 Avg fitness: -24.462996826635422
Generation 3 Max fitness: -0.268343921365924 Min fitness: -48.268732539710875 Avg fitness: -23.40960795742299
Generation 4 Max fitness: -0.268343921365924 Min fitness: -44.816465075409944 Avg fitness: -22.40145487982277
Generation 5 Max fitness: -0.268343921365924 Min fitness: -42.3065554500819 Avg fitness: -21.553857070051322
Generation 6 Max fitness: -0.268343921365924 Min fitness: -40.54360020827952 Avg fitness: -20.76267893376614
Generation 7 Max fitness: -0.268343921365924 Min fitness: -39.059801739417935 Avg fitness: -19.984161605799528
Generation 8 Max fitness: -0.268343921365924 Min fitness: -37.53433342355018 Avg fitness: -19.238066962962293
Generation 9